In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from os import listdir
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16 
%matplotlib inline

In [5]:
def crop_brain_contour(image, plot=False):
    
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Preprocessed Image')
        
        plt.show()
    
    return new_image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vggmodel = load_model("/content/drive/My Drive/VGG_model.h5")


In [19]:
path="/content/drive/My Drive/augmented_data/no/aug_28 no_0_5806.jpg"
img=cv2.imread(path)
img = crop_brain_contour(img, plot=False)
img = cv2.resize(img, dsize=(256,256), interpolation=cv2.INTER_CUBIC)
img = img / 255.
from keras.preprocessing import image
test_image = image.img_to_array(img)
test_image = np.expand_dims(test_image, axis = 0)

In [20]:
test_f=vggmodel.predict(test_image)
test_f=test_f.reshape(test_f.shape[0],-1)

In [22]:
import pickle
from sklearn import svm
filename = '/content/drive/My Drive/finalized_model.sav' 
# load the model from disk
svm_model = pickle.load(open(filename, 'rb'))

prediction_svm=svm_model.predict(test_f)
if(prediction_svm[0]==0):
   print("No tumor detected")
else:
   print("Tumor detected")


No tumor detected
